In [ ]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import collect_segments
from util import download_audio_gdrive
from util import split_audio_files

In [ ]:
df = read_spreadsheet(sheet_id = "1yKSzConuVWo8BuMDs2mabF5iiBKUz2wF--LIabFN6QE")

for index, row in df.iterrows():
    file_name = row['ID']
    gd_url = row['Audio link']
    Sr_no = row['z']
    
    # if not isinstance(gd_url, str) or not isinstance(id, str):
    #     continue
    if Sr_no >= 596 and Sr_no <= 644:
        print(file_name, gd_url, Sr_no)
        file_name = file_name + 'mp3'
        download_audio_gdrive(gd_url, file_name)


In [ ]:
split_audio_files('STT_AB', 'mp3')

In [ ]:
collect_segments('STT_AB', 'after_split', 'segments_ab')

In [ ]:
! aws s3 cp segments_ab  s3://monlam.ai.stt/stt_pecha_tools/ --recursive

In [ ]:
target_directory = f"segments_ab"

In [ ]:
import os
def getTimeSpan(filename):
    filename = filename.replace(".wav", "")
    filename = filename.replace(".WAV", "")
    filename = filename.replace(".mp3", "")
    filename = filename.replace(".MP3", "")
    try:
        if "_to_" in filename:
            start, end = filename.split("_to_")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return (end - start)/1000
        else:
            start, end = filename.split("-")
            start = start.split("_")[-1]
            end = end.split("_")[0]
            end = float(end)
            start = float(start)
            return abs(end - start)
    except Exception as err:
        print(f"filename is:'{filename}'. Could not parse to get time span.")
        return 0
    
getTimeSpan("STT_AB00094_0057_290646_to_295860.mp3")
getTimeSpan("STT_AB00094_0057_290646_to_295860")

In [ ]:
import os
rows = []
from transformers import pipeline
from tqdm.auto import tqdm
generator = pipeline(model="openpecha/wav2vec2_run8")

for root, _, files in os.walk(target_directory):
        for file in tqdm(files):
            source_file_path = os.path.join(root, file)
            opt = generator(f"/media/monlamai/SSD/GitHub/split/STT_AB/{target_directory}/{file}")
            inf = opt['text']
            # print(inf)
            rows.append([file.replace('.wav', ''), f"https://d38pmlk0v88drf.cloudfront.net/stt_pecha_tools/{file}", inf , getTimeSpan(file)])

In [ ]:
import pandas as pd
df = pd.DataFrame(rows, columns =['file_name', 'url', 'inference_transcript', 'audio_duration'])

In [ ]:
df.head()

In [ ]:
df[['inference_transcript','url']].iloc[0:10].to_dict()

In [ ]:
df.to_csv(f"stt_ab_from_gdrive.tsv", index=False, sep="\t")